In [1]:
!date

Mon Mar 22 13:49:10 EDT 2021


# Backward time loop to track coast-crossing vapor lakes (CCVLs)

------------


## each new assigned CCVL creates a folder called `tag/` 

### tag format is `yyyymmddhh.meanlat`
    meanlat is the mean latitude of the overlap with a coastline or (simplest case) meridian
    
## tag/shapefiles/
    contains a set of files yymmddhh.contours

## tag/scratch.txt 
    used during loop (or can be an overwriteable memory object, but a file feels clearer right now)

## tag/times.txt    
    one line for each time level when CCVL `tag` exists: [time, nsegments, total-area, bbox]

In [1]:
import sys
import os
import xarray as xr
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

/Users/bem/miniconda3/envs/unidata/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


ModuleNotFoundError: No module named 'dask'

----------
# Time loop over hours, backward in time


In [ ]:
for time in reverse(alltimes): 
    yymmddhh = format(time)
    
    # create an empty dictionary for all the CCVLs that survive to the next iteration (the next-earlier time level)
    CCVLs_next = {}

## Loop over all connected spatial segments in "time zero" array, CWV0

a newly defined image or map **segment** is an object with 

1. an integer mask label within its CWV array or image (assigned by the segmentation)
2. room for an attribute called a *tag* (a CCVL tag, format as above) -- OHH MAYBE MANY? DAMMIT

#### We only want to proceed if seg0 doesn't touch the domain boundary

In [ ]:
    CWV0 = read(this time level)
    
# Need a function called **segment** that returns all the segments, and their maximum longitude (perhaps other properties too)
    segments,maxlons = segment(CWV0)
    segflags = segments.copy()*0 # a set of flags all initially zero, later set to 1 if the segement overlaps an existing CCVLs_active

# "inbounds" = just the ones not touching the edge of the domain   
    inbounds = segments.where(maxlons lt 100)
    

## Memory items available: 

In working memory within the loop are always two arrays **CWV1(x,y) and CWV0(x,y)**. 
CWV1 is at the previous iteration (but chronologically later time), while CWV0 is presently being processed as "time zero". Both are segmented and contoured at the selected threshold value (say 55 mm). 

**Segments** (pixel clusters or contours) can be input to a routine that evaluates their spatial overlap. A coastline segment (shapefile or pixel mask) is also available to evaluate overlap. 

**CCVLs_active** is a dictionary of lists, structured like this: 

{

tag1:[segment7, segment8, segment9, ...] 

tag2:[segment23, segment24, ...]

...

}

# Loop over the tags in CCVLs_active, 
## and for each of those loop over all of its segments, 
### and for each of those loop over all inbounds segments seg0 

In [ ]:
    for CCVLkey, CCVLlist in CCVLs_active.items():

# define string tag
        tag = CCVLkey
# initialize tagstats_t0
        tagstats_t0 = [0,0,0.0,-999,999,-999,999] # nsegs, npixels, totcwv, Eedge, Wedge, Nedge, Sedge
    
        for seg1 in CCLlist: 
            for iseg, seg0 in enumerate(inbounds, start=0): 

## OK, we have iterates seg0, and seg1
### Do they overlap? 
### If so, then CCVL_next contains seg0

In [ ]:
                if overlap(seg1, seg0):
                    # consequence 1: this CCVL remains active (so CCVLkey belongs in the CCVL_new dictionary for next time iteration)
                    # that dictionary's value is a list of segments within CWV0, so syntax (perhaps illegal) is:
                    CCVL_new.update( {CCVLkey : append(seg0)} )  
                    
                    # consequence 2: seg0 is not eligible to pioneer a new CCVL. Need to mark it somehow. 
                    segflags[iseg] = True

-------------
----------------
## Strategize the desired outputs, so we can write results as they are obtained:

----------------
## The use of the output will be to 

    * screen whole-lifetime CCVL events based on lines in `CClakes.txt`
    * For selected events, further screen the hourly objects, based on lines in `tag/tag.times.txt`
    * Visualize lakes at selected times by placing shapefile contours on a geographical map, perhaps color-coded by time or by tag
-----------------
-----------------

### Therefore, at this step, we need to append seg0 as a shape in `tag/shapefiles/yymmddhh.contours`
### and also tally up seg0's contribution to the bulk statistics of CCVLkey at this time zero 



In [ ]:
                    contour_append(tag + '/shapefiles/' + yymmddhh + '.contours', seg0)  
                    update_1timetagstats( tagstats, seg0 ) # nsegs, npixels, kgwater, max/min lat/lon

#### End of loop over all seg0, and all seg1 for a given CCVL. If tagstats_t0[0] is still its initialization of zero, the CCVL quietly ends. 

In [ ]:

                # end for seg0 in ibounds            
            # end for seg1 in CCVLlist
            if (tagstats_t0[0] > 0): 
                append_1timetagstats(tagstats, tag+'/'+yymmhhdd+'.txt')

# what if some seg0 overlaps the coast, and not any previous seg0? 
# A new CCVL! 

In [ ]:
for iseg, seg0 in enumerate(inbounds, start=0): 
    if( does_overlap(coastline, seg0) and segflags[iseg] == False):
        
# Create a new 'tag' based on time and the latitude of the coast-crossing segment
        lala = meanlat_of_overlap(coastline,seg0)
        newtag = yymmddhh + '_' + str(lala)
        CCVL_new.update( {CCVLkey : append(seg0)} )  


### end of time zero (t0). Close the time loop and iterate. 

In [ ]:
# end for time in reverse(alltimes): 
# Overwrite the dictionary of active CCVLs for the next iteration 
CCVLs_active = CCVLs_new
# END CODE: ITERATION BACKWARD OVER TIME NOW REPEATS

-----------------
-----------------
-----------------

# a SEPARATE code can then glob over all the tag/yymmhhdd.txt files and construct `CClakes.txt`
## with one line per tag, comprising the stats over the whole lifetime of each CCVL
### summing up start and end date, lifetime in hours, total (area x hours), total (vapor x hours), bounding box, centroid, etc. etc. 

#### All times (and thus tags and filenames) are based on the time of *last* contact with the coastline (first encountered, in the reverse-time flow of the algorithm's time loop). 

# Open the data file of all time slices 

In [6]:
!ls -atlh /data2/brian/WEIO_30-100_20S-20N_2014-8.nc

-rw-rw-r-- 1 bmapes bmapes 755M Mar 10 18:55 /data2/brian/WEIO_30-100_20S-20N_2014-8.nc


In [7]:
TQV_WEIO = xr.open_dataset('/data2/brian/WEIO_30-100_20S-20N_2014-8.nc')
TQV_WEIO

<xarray.Dataset>
Dimensions:  (lat: 81, lon: 113, time: 43824)
Coordinates:
  * lat      (lat) float64 -20.0 -19.5 -19.0 -18.5 -18.0 ... 18.5 19.0 19.5 20.0
  * lon      (lon) float64 30.0 30.62 31.25 31.88 ... 98.12 98.75 99.38 100.0
  * time     (time) datetime64[ns] 2014-01-01 ... 2018-12-31T23:00:00
Data variables:
    TQV      (time, lat, lon) float32 ...

In [4]:
# playing with dictionaries for syntax test 
str1 = '2020030123'
str2 = 'dog'

dict = {}
dict.update({0.34 : str2})

dict
# print(dict[str1])


{0.34: 'dog'}